<a href="https://colab.research.google.com/github/StevenBryceLee/AdvancedDataStructures/blob/master/module1-rnn-and-lstm/Copy_of_LS_DS_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [ ]:
import requests
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import LambdaCallback
import random
import sys

In [ ]:
url = "https://www.gutenberg.org/files/100/100-0.txt"

r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text
data = data.split('\r\n')
toc = [l.strip() for l in data[44:130:2]]
# Skip the Table of Contents
data = data[135:]

# Fixing Titles
toc[9] = 'THE LIFE OF KING HENRY V'
toc[18] = 'MACBETH'
toc[24] = 'OTHELLO, THE MOOR OF VENICE'
toc[34] = 'TWELFTH NIGHT: OR, WHAT YOU WILL'

locations = {id_:{'title':title, 'start':-99} for id_,title in enumerate(toc)}

# Start 
for e,i in enumerate(data):
    for t,title in enumerate(toc):
        if title in i:
            locations[t].update({'start':e})
            

df_toc = pd.DataFrame.from_dict(locations, orient='index')
df_toc['end'] = df_toc['start'].shift(-1).apply(lambda x: x-1)
df_toc.loc[42, 'end'] = len(data)
df_toc['end'] = df_toc['end'].astype('int')

df_toc['text'] = df_toc.apply(lambda x: '\r\n'.join(data[ x['start'] : int(x['end']) ]), axis=1)

In [ ]:
#Shakespeare Data Parsed by Play
df_toc.head()

title  ...                                               text
0  THE TRAGEDY OF ANTONY AND CLEOPATRA  ...                                                   
1                       AS YOU LIKE IT  ...  AS YOU LIKE IT\r\n\r\n\r\nDRAMATIS PERSONAE.\r...
2                 THE COMEDY OF ERRORS  ...  THE COMEDY OF ERRORS\r\n\r\n\r\n\r\nContents\r...
3            THE TRAGEDY OF CORIOLANUS  ...  THE TRAGEDY OF CORIOLANUS\r\n\r\nDramatis Pers...
4                            CYMBELINE  ...  CYMBELINE.\r\nLaud we the gods;\r\nAnd let our...

[5 rows x 4 columns]

In [ ]:
df_toc.title.value_counts()

THE LIFE OF KING HENRY THE FIFTH            1
THE TRAGEDY OF ANTONY AND CLEOPATRA         1
VENUS AND ADONIS                            1
THE LIFE OF TIMON OF ATHENS                 1
THE HISTORY OF TROILUS AND CRESSIDA         1
THE LIFE OF KING HENRY V                    1
THE WINTER’S TALE                           1
OTHELLO, THE MOOR OF VENICE                 1
THE FIRST PART OF KING HENRY THE FOURTH     1
MUCH ADO ABOUT NOTHING                      1
MACBETH                                     1
THE PASSIONATE PILGRIM                      1
THE THIRD PART OF KING HENRY THE SIXTH      1
LOVE’S LABOUR’S LOST                        1
THE RAPE OF LUCRECE                         1
THE TRAGEDY OF MACBETH                      1
THE TRAGEDY OF OTHELLO, MOOR OF VENICE      1
KING RICHARD THE THIRD                      1
THE TRAGEDY OF JULIUS CAESAR                1
THE MERRY WIVES OF WINDSOR                  1
AS YOU LIKE IT                              1
KING HENRY THE EIGHTH             

In [ ]:
text = df_toc['text'][df_toc['title'] == 'THE MERCHANT OF VENICE'].values[0]
text

'THE MERCHANT OF VENICE\r\n\r\n\r\n\r\nContents\r\n\r\nACT I\r\nScene I. Venice. A street.\r\nScene II. Belmont. A room in Portia’s house.\r\nScene III. Venice. A public place.\r\n\r\n\r\nACT II\r\nScene I. Belmont. A room in Portia’s house.\r\nScene II. Venice. A street.\r\nScene III. The same. A room in Shylock’s house.\r\nScene IV. The same. A street.\r\nScene V. The same. Before Shylock’s house.\r\nScene VI. The same.\r\nScene VII. Belmont. A room in Portia’s house.\r\nScene VIII. Venice. A street.\r\nScene IX. Belmont. A room in Portia’s house.\r\n\r\n\r\nACT III\r\nScene I. Venice. A street.\r\nScene II. Belmont. A room in Portia’s house.\r\nScene III. Venice. A street.\r\nScene IV. Belmont. A room in Portia’s house.\r\nScene V. The same. A garden.\r\n\r\n\r\nACT IV\r\nScene I. Venice. A court of justice.\r\nScene II. The same. A street.\r\n\r\n\r\nACT V\r\nScene I. Belmont. The avenue to Portia’s house.\r\n\r\n\r\n\r\nDramatis Personæ\r\n\r\nTHE DUKE OF VENICE\r\nTHE PRINCE OF M

In [ ]:
chars = list(set(text))
print ('{} unique characters'.format(len(vocab)))
char_int = {u:i for i, u in enumerate(vocab)}
int_char = {i:c for i, c in enumerate(chars)} 

74 unique characters


In [ ]:
textlength = len(df_toc['text'][df_toc['title'] == 'THE MERCHANT OF VENICE'].values[0])
textlength

125434

In [ ]:
maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = []
next_char = []

for i in range(0, len(encoded) - maxlen, step):
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  25079


In [ ]:
x = np.zeros((len(sequences), maxlen, len(vocab)), dtype=np.bool)
y = np.zeros((len(sequences),len(vocab)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
    y[i, next_char[i]] = 1

In [ ]:
model = Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(vocab))))
model.add(layers.Dense(len(vocab), activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    print(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
    print(sentence)


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
history = model.fit(x, y, batch_size = 32, epochs=50, callbacks=[print_callback])

Epoch 1/50
784/784 [==============================] - ETA: 0s - loss: 2.4034
----- Generating text after Epoch: 0
----- Generating with seed: " with the cause in controversy between t"
.’sæEcMæææprGJMG[GacMdFpEcMJæEnoaEoææ’EJ’EfærOTsMMs”G.’FæMMææ”æMMEa
784/784 [==============================] - 49s 62ms/step - loss: 2.4034
Epoch 2/50
784/784 [==============================] - ETA: 0s - loss: 2.2403
----- Generating text after Epoch: 1
----- Generating with seed: "with ornament.
In law, what plea so tai"
with ornament.
.’pæPLJMWGMkEkaa_zM,æ”ELæWFæEæM”æ_MJM_ctGG
784/784 [==============================] - 50s 63ms/step - loss: 2.2403
Epoch 3/50
784/784 [==============================] - ETA: 0s - loss: 2.1353
----- Generating text after Epoch: 2
----- Generating with seed: "n’s son”—or rather an honest woman’s son"
.’
.’mæ”æhMMzGFJMFRGæ’_”ææ’œMPæeEnzJæBæQ”sææhQfæææEæ’__æ”æ’LJMaFcJ”’PQy”’cJF[æfQR’P”æEa__M”æF[”fG:”æ’”æRJM’_yæMGæJF[hMJMG”HG[hMGJFMJMYrMPcGQFaaænæœ_WFJMæMMMGGæPJrQfFJMYFG[sMG[M

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN

https://colab.research.google.com/drive/1GvMnNqe0XPm84qrspBinO84dIVVNl8gP#scrollTo=Gd7xY7-IgTLH